In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mutual_info_score
from sklearn.model_selection import train_test_split
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score 

In [3]:
df1=pd.read_excel("https://query.data.world/s/eyrdmwczqapeywqo3bo2tpen6dqntz",header=1)
df1.drop(252,inplace=True)
for i in df1.columns:
    if i[:7]=="Unnamed":
        df1.rename(columns={i:j+" "+str(k)},inplace=True)
        k=k+1
    else:
        j=i
        k=1
growth_percent=(df1["C2) Funding FY2009"].sum()-df1["C1) Funding FY2008"].sum())*100/df1["C1) Funding FY2008"].sum()
growth_percent

19.935203400708026

In [4]:
def target_value(y):
    y[y>=0]=1
    y[y<0]=0
    return y

In [5]:
df1["y"]=target_value((df1["C2) Funding FY2009"]-df1["C1) Funding FY2008"])/df1["C1) Funding FY2008"])
df1["y"].fillna(2,inplace=True)
df1=df1[df1["y"]!=2]
y1=df1["y"].values
df1.drop("y",axis=1,inplace=True)
y1

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,
        1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,
        0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,
        1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1

In [6]:
def mutual(df2,y):
    lt=[]
    for i in df2.columns:
        j=df2[i]
        lt.append(mutual_info_score(j.values,y))
    mi_score=pd.DataFrame(lt,columns=["mutual info score"])
    mi_score["column name"]=df2.columns
    mi_score.set_index("column name",inplace=True)
    return mi_score
df1.fillna(0,inplace=True)
for i in df1.columns[:6]:
    j=df1[i]
    j.replace(df1[df1[i]!=0][i].unique(),range(1,len(df1[df1[i]!=0][i].unique())+1),inplace=True)
for i in df1.columns[9:]:
    j=df1[i]
    j.replace(df1[df1[i]!=0][i].unique(),range(1,len(df1[df1[i]!=0][i].unique())+1),inplace=True)    
pd.concat([mutual(df1.iloc[:,0:6],y1),mutual(df1.iloc[:,9:],y1)])

,mutual info score
column name,
Index Number,0.543095
Investment Name,0.543095
Agency,0.045957
Subagency,0.182701
A) Brief Description,0.543095
B) Year Established,0.099016
D) Mission-specific or General STEM?,0.002525
E) Agency or Mission-Related Workforce Needs?,0.005587
F1) Primary Investment Objective,0.014423


In [95]:
#train_test_split
xgb = XGBClassifier()
xgb.fit(df1, y1)
# plot feature importance
#plt.figure(figsize=(10,20))
#plot_importance(model)
#plt.show()
df2=pd.DataFrame(xgb.feature_importances_,index=df1.columns)
X_train,X_test,y_train,y_test=train_test_split(df1[df2[df2[0]>0.04].index],y1,test_size=0.3,random_state=57)

In [96]:
#XGBClassifier
#gs=GridSearchCV(XGBClassifier(),param_grid={"n_estimators":[10,100,500,1000],"max_depth":[3,4,5,6],"learning_rate":[0.01,0.1,1,2.5]},
#               scoring="roc_auc")
#gs.fit(X_train,y_train)
#gs.best_params_
xgb = XGBClassifier(learning_rate=2,max_depth=6,n_estimators=500)
xgb.fit(X_train,y_train,eval_metric='auc')
y_pred = xgb.predict(X_test)
roc_auc_score(y_test,y_pred)

0.88342318059299196

In [5]:
import pytest
def test_case():
    assert target_value([2,4.5,-.06])==[1,1,0]